# DENAS for Compute Vision 


## Step1: Env setup


conda activate pytorch_1.10

## Step2: Configuration setup for DENAS


In [1]:
aidk_denas_conf = {"zero_shot_score": "De_score2",
"search_space": "SearchSpace/search_space_XXBL.py",
"budget_model_size": "1000000",
"budget_flops": "10000000",
"max_layers": "18",
"batch_size": "64",
"input_image_size": "32",
"plainnet_struct_txt": "SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1)",
"num_classes": "100", 
"evolution_max_iter": "1004",
"population_size": "10",
"save_dir": "./DENAS_log",
"no_reslink": "False",
"no_BN": "False",
"use_se": "False",
"budget_latency": "1"}

In [2]:
import sys
sys.path.append("..") 
from DeNas.denas import configure, parse_args
sys.argv = ["--max_search_iter", "104", "--conf","../conf/aidk_denas_cv.conf"]
input_args, unknown_args = parse_args(sys.argv[0:])
settings = configure(input_args, unknown_args)

/home/vmagent/app/hydro.ai/demo/..
/home/vmagent/app/hydro.ai/demo/../DeNas
fail to import hvd.
{'domain': 'cv', 'max_search_iter': 104, 'budget_model_size': 1000000, 'budget_flops': 10000000, 'budget_latency': 1, 'batch_size': 64, 'save_dir': './DENAS_log', 'conf': '../conf/aidk_denas_cv.conf', 'log': 'INFO', 'zero_shot_score': 'De_score2', 'search_space': 'SearchSpace/search_space_XXBL.py', 'max_layers': 18, 'input_image_size': 32, 'plainnet_struct_txt': 'SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1)', 'num_classes': 100, 'evolution_max_iter': 2004, 'population_size': 10, 'no_reslink': False, 'no_BN': False, 'use_se': False}


In [3]:
# compute network zero-shot proxy score
def _compute_nas_score(self, random_structure_str):
    the_model = self.main_net(num_classes=self.num_classes, plainnet_struct=random_structure_str, no_create=False, no_reslink=True)

    if self.zero_shot_score == "De_score_base":
        nas_core = compute_de_score.do_compute_nas_score(model=the_model, gpu=None,
                                                                   resolution=self.input_image_size,
                                                                   batch_size=self.batch_size,
                                                                   mixup_gamma=1e-2)
    elif self.zero_shot_score == "De_score_transformer":
        nas_core = compute_de_score_plus.do_compute_nas_score(model=the_model, gpu=None,
                                                                   resolution=self.input_image_size,
                                                                   batch_size=self.batch_size,
                                                                   mixup_gamma=1e-2)
    return nas_core

## Step3: Launch Search on AIDK DENAS framework  

In [4]:
from DeNas.cv.third_party.ZenNet import DeSearchSpaceXXBL as DeSearchSpace
from DeNas.cv.third_party.ZenNet import DeMainNet as DeMainNet
from DeNas.denas import DeNasEASearchEngine

searcher = DeNasEASearchEngine(main_net = DeMainNet, search_space = DeSearchSpace, settings = settings)
searcher.launch()
best_structure = searcher.get_best_structures()

print(f"DeNas search completed, best structure is {best_structure}")


loop_count=10/104, max_score=0.0326998, min_score=0.000153411, time=0.000331521h
loop_count=20/104, max_score=0.0455763, min_score=0.000197802, time=0.000670239h
loop_count=30/104, max_score=0.0455763, min_score=0.000197802, time=0.00109327h
loop_count=40/104, max_score=0.0552625, min_score=0.0001252, time=0.00155366h
loop_count=50/104, max_score=0.0552625, min_score=0.0001252, time=0.00208839h
loop_count=60/104, max_score=0.0552625, min_score=0.0001252, time=0.00229916h
loop_count=70/104, max_score=0.0552625, min_score=0.0001252, time=0.00262857h
loop_count=80/104, max_score=0.0552625, min_score=0.0001252, time=0.00295829h
loop_count=90/104, max_score=0.0552625, min_score=0.0001252, time=0.00339657h
loop_count=100/104, max_score=0.0552625, min_score=0.0001252, time=0.00365788h
DeNas search completed, best structure is [(tensor(0.0553), 0.000125199556350708, 'SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)

## Step4: Save the best network architecture

In [5]:
#save the best model structure
from DeNas.utils import global_utils
import os
best_structure_txt = os.path.join(settings["save_dir"], 'best_structure.txt')
global_utils.mkfilepath(best_structure_txt)
with open(best_structure_txt, 'w') as fid:
        fid.write(best_structure[0][2])
print(f"Best structure saved on the path:{best_structure_txt}")

Best structure saved on the path:./DENAS_log/best_structure.txt


## Step5:  Load, build and Training with DE-NET


In [6]:
# Train on CIFAR10 dataset with 100 iterations
from DeNas.cv.train_image_classification import train
train()


fail to import hvd.
loaded model, training start


````diff
Files already downloaded and verified
Files already downloaded and verified
loading model:
optimizer is :
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.025
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    lr: 0.025
    momentum: 0.9
    nesterov: True
    weight_decay: 0
)
--- Start training epoch 0
Create model: Masternet.py:MasterNet
/home/xianyang/tmp/DENAS/frameworks.bigdata.bluewhale/DeNas/cv/train_image_classification.py:232: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return input.mul_(gamma).add_(1 - gamma, perm_input), target.mul_(gamma).add_(1 - gamma, perm_target)
Train epoch=0, i=0, loss=3.2746, acc1=12.5%, acc5=51.5625%, lr=3.25e-05
Eval epoch=0, i=0, loss=157.172, acc1=3.125%, acc5=42.1875%
Eval epoch=0, i=100, loss=134.759, acc1=12.5%, acc5=57.8125%
 * Validate Acc@1 9.880 Acc@5 51.540, n_val=10000

Train time:5.59848403930664
````

In [7]:
# system configuration
# CPU: Intel(R) Xeon(R) Platinum 6240 CPU @ 2.60GHz
# NIC: Mellanox ConnectX-4
# Storage - boot: 1x INTEL SSDSCKKB24 OS Drive
# Storage - application drives: INTEL SSDPE2KX010T8 for data storage
# OS: CentOS Linux 7 (Core)
# Kernel: 3.10.0-1160.49.1.el7.x86_64

## Performance comparsion

<font size=3>
    
- 8.3x speedup over resnet101 with 70x parameter reduction,1.1x accuracy improvement 
Better than SOTA work including Zen-Net & Synflow 
- Top-1 accuracy: 0.96 vs. 0.95 vs. 0.95
    
- NAS search time: 9h vs. 21h vs. 13h
</font>

<img src="./img/result2.png" width="50%">
<center>DE_Net vs SOTA methods</center>

<font size=3>
    
DE_Net VS ResNet50

- 4.23x training speedup with 1.001x better accuracy 
- 3.9x less FLOPS, 39x less parameters 

</font>

<img src="./img/result_cnn.PNG" width="50%">
<center>DE_Net vs ResNet50</center>

<font size=3>
    
DE_Net VS Autoformer

- 38.36x speedup for searching time
- 1.62x training speedup, 1.79x speedup for test  
- 3.86x less parameters size


</font>

<img src="./img/transformer1.png" width="50%">
<center>DE_Net vs Autoformer search</center>


<img src="./img/transformer2.png" width="50%">
<center>DE_Net vs Autoformer train</center>


## NAS-Bench -201 Performance results

Usage Case

* Task: Image Classification
* Dataset:CIFAR10 & CIFAR100
* Metric: Spearman

NAS–bench-201
* Generate cdd architecture – 5^6
* A table with pre-calculated train, test and validate accuracy 
* Then NAS-bench compare spearman correlations with SOTA zero-cost, including synflow, jacbi_cov, zen_score, grad_now Spearman’s rank correlation coefficient between final validation accuracy and zero-cost proxy 

DE-NAS
* Adding DE_Score into SOTA as comparison, our train free DE-Score show better result than SOTA NAS

<img src="./img/nas_bench_result.png" width="50%">
<center>NAS-Bench-201 Spearman results</center>


# Next Step work 
<font size=4>
    
- Integrated DE-NAS with HuggingFace
- Continue optimize ViT trainer



</font>

## Reference 

1. NAS-Bench-201: https://arxiv.org/abs/2001.00326
2. AutoFormer: Searching Transformers for Visual Recognition: https://arxiv.org/abs/2107.00651
3. Training-free Transformer Architecture Search: https://arxiv.org/pdf/2203.12217.pdf
4. LiteTransformerSearch: Training-free On-device Search for Efficient Autoregressive Language Models: https://arxiv.org/pdf/2203.02094.pdf 